# （1）赛题介绍

图神经网络（Graph Neural Network）是一种专门处理图结构数据的神经网络，目前被广泛应用于推荐系统、金融风控、生物计算等领域。图神经网络的经典问题主要有三类，分别为节点分类、连接预测和图分类。本次比赛旨在让参赛同学了解并掌握如何使用图神经网络处理节点分类问题。

在过去的一个世纪里，科学出版物的数量每12年增加近一倍，对每一种出版物的主题及领域进行自动分类已成为当下十分重要的工作。本次任务的目标是预测未知论文的主题类别，如软件工程，人工智能，语言计算和操作系统等。比赛所选35个领域标签已得到论文作者和arXiv版主确认并标记。

本次比赛选用的数据集为arXiv论文引用网络——ogbn-arixv数据集的子集。ogbn-arixv数据集由大量的学术论文组成，论文之间的引用关系形成一张巨大的有向图，每一条有向边表示一篇论文引用另一篇论文，每一个节点提供100维简单的词向量作为节点特征。在论文引用网络中，我们已对训练集对应节点做了论文类别标注处理。本次任务希望参赛者通过已有的节点类别以及论文之间的引用关系，预测未知节点的论文类别。

[赛题地址](https://aistudio.baidu.com/aistudio/competition/detail/59)

# （2）思路介绍+个人方案亮点
1、这个赛题难点主要是容易过拟合，准确率提高困难，另外还有就是aistudio内存小，"num_layers" ，"hidden_size"，"heads"等参数如果调太大的话内存会爆掉。所以调模型结构及超参数会变得很困难。

2、每次运行时，验证集准确率都要高于训练集数据，分析是模型欠拟合，但调超参往往会爆掉内存，所以"num_layers" ，"hidden_size"，"heads"等参数改动较小。

3、合适的调学习率可以加快模型训练速度，可以使用学习率衰减。

4、在模型训练后期主要表现为过拟合，为了减少过拟合，采用了减少特征维度、主成份分析等方法。

5、为了提高训练速度和准确率，采用了增加标签节点和在feat中增加标签的方法。

6、采用投票方法提高测试集准确率。

# （3）具体方案分享

## 环境配置
### 环境配置就入坑，以前下方代码就能安装pgl 和easydict,现在新建项目运行以下代码就会报错：

!mkdir /home/aistudio/external-libraries
!pip install pgl easydict -q -t /home/aistudio/external-libraries
!mkdir /home/aistudio/external-libraries

### 现在新建项目需运行以下代码才不会报错：

!pip install chardet==3.0.4  
!pip install easydict -q
!pip install pgl

## 模型选择及学习率调整
这里已经有很多强大的模型配置了，你可以尝试简单的改一下config的字段。 例如，换成GAT的配置


config = {
    "model_name": "GAT",
    "num_layers":  1,
    "dropout": 0.5,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "edge_dropout": 0.00,
}


个人感觉用GAT模型效果效好，可以用ResGAT版的，对层数、学习率、droupout等参数进行调整。
ResGAT主要代码：

        feature=L.fc(feature,size=self.hidden_size,name="init_feature")
        res_feature=feature

        
        feature=feature+res_feature
        feature=L.relu(feature)
        feature=L.layer_norm(feature,name="ln_%s" % i)


这个项目学习率对训练速度影响很大，建议使用学习率衰减。
学习率衰减可以在build_model.py中调整，代码大致如下，按照自己的模型修改参数，多试几次，或用其它的学习率衰减方法。

    boundaries = [600，800]
    values = [0.001, 0.0001，0.00001]
    learning_rate=fluid.layers.piecewise_decay(boundaries=boundaries, values=values)



## 调参经验

### 1.特征调整
（1）把特征维度从100减少到80或者50等，但是感觉效果变差了。
####  简单粗暴的读取node_feat 前80维特征。
（2）用主成分分析，保留了约99%的特征。这个似乎可能有一点效果，不是很确定。
#### 用主成分分析主要代码如下：

```
def data_preprocess(X):
    """
    数据归一化
    :param X: ndarray,原始数据
    :return: (ndarray.ndarray,ndarray),处理后的数据,每个特征均值，每个特征方差
    """
    mean = np.mean(X, axis=0,dtype='float32')
    std = np.std(X, axis=0, ddof=1,dtype='float32')  # 默认ddof=0, 这里一定要修改
    return (X - mean) / std, mean, std
def pca(X):
    sigma = X.T.dot(X) / len(X)  # (n,m)x(m,n) (n,n)
    u, s, v = np.linalg.svd(sigma)  # u(n,n) s(n,), v(n,n)
    return u, s, v
def project_data(X, U, K):
    """
    数据降维
    :param X: ndarray,原始数据
    :param U: ndarray,奇异值分解后的U
    :param K: int,目标维度
    :return: ndarray,降维后的数据
    """
    return X.dot(U[..., :K]).astype('float32')
def reconstruct_data(Z, U, K):
    """
    数据升维
    :param Z: ndarray,降维后的数据
    :param U: ndarray,奇异值分解后的U
    :param K: int,降维的维度
    :return: ndarray,原始数据
    """
    return Z.dot(U[..., :K].T)
def normalization(data):
    _range = np.max(data,axis=0) - np.min(data,axis=0)
    return (data - np.mean(data,axis=0)) / _range
```

### 2.提前中止。
为了减少过拟合，在验证集上准确率较高时提前终止训练。这个可以用手动中止的。
### 3.小技巧：
epoch完成或中止后，想继续运行，用#把exe.run(startup_program)注释掉，继续运行。

如果想重头开始，则需重启执行器，此时记得把#exe.run(startup_program)前面的#去掉，否则会报错。

## 增加标签节点
先训练一个70+的结果，对每个节点预测一个标签，然后对每个节点增加标签节点，同时扩展feat，在原feat后加上标签，并加上相应的边，重新训练，不断循环。
 
在训练集和验证集上都加上标签节点后，很容易跑飞，验证集轻松上80+，可以多试几次，看测试集效果，最后投票提升效果。

代码详见 label_add.py 。

## 投票集成
简单投票集成学习，这个可以提升效果，尽量选得分高的进行投票。


在统计学和机器学习中，集成学习方法使用多种学习算法来获得比单独使用任何单独的学习算法更好的预测性能。

使用不同超参数、不同的特征，不同的结构，运行多次模型可得到不同的预测结果。在这里我使用的是简单投票法，即选用得票数最多的分类结果作为最终的预测结果。
预测出多个submission.csv后，进行投票，代码如下：

```
def publicnum(num, d = 0):
    dictnum = {}
    for i in range(len(num)):
        if num[i] in dictnum.keys():
            dictnum[num[i]] += 1
        else:
            dictnum.setdefault(num[i], 1)
    maxnum = 0
    maxkey = 0
    for k, v in dictnum.items():
        if v > maxnum:
            maxnum = v
            maxkey = k
    return maxkey
    
df1=pd.read_csv("submission1.csv")
df2=pd.read_csv("submission2.csv")
df3=pd.read_csv("submission3.csv")
df4=pd.read_csv("submission4.csv")
df5=pd.read_csv("submission5.csv")
df6=pd.read_csv("submission6.csv")
df7=pd.read_csv("submission7.csv")

nids=[]
labels=[]

for i in range(df4.shape[0]):
    label_zs=[]
    label_zs.append(df1.label[i])
    label_zs.append(df2.label[i])
    label_zs.append(df3.label[i])
    label_zs.append(df4.label[i])
    label_zs.append(df5.label[i])
    label_zs.append(df6.label[i])
    label_zs.append(df7.label[i])
    lab=publicnum(label_zs, d = 0)
    labels.append(lab)
    nids.append(df4.nid[i])
    
submission = pd.DataFrame(data={
                            "nid": nids,
                            "label": labels
                        })
submission.to_csv("submission.csv", index=False)
```

# （4）总结及改善方向
1. 因为内存小，调超参要小心，否则容易爆内存。（某些参数有时会爆内存，有时不会，看人品）。
2. 选择好的模型有助于提高训练成绩。
3. 选择合适的学习率有利于加快训练过程。
4. 减少特征维度用于训练，效果有待进一步验证。
5. 提前中止有利于提高成绩。
6. 增加标签节点，有利于提高训练集和验证集准确率，但无法有效把握中止时机，十分容易过拟合，需进一步优化。



# （5）飞桨使用体验+给其他选手学习飞桨的建议
###  以前没怎么关注过深度学习，接触飞桨后，逐步了解了深度学习，深度学习从飞桨开始，在深度学习的道路上，飞桨伴我一路成长，希望飞桨越来越好。

###  建议大家多参加百度AI Studio课程，多看别人写的AI Studio项目，也许会有灵感迸发，在比赛中取得更好的成绩。

# （6）One More Thing
如果大家还想要别的奇思妙想，可以参考以下论文，他们都在节点分类上有很大提升。

Predict then Propagate: Graph Neural Networks meet Personalized PageRank (https://arxiv.org/abs/1810.05997)

Simple and Deep Graph Convolutional Networks (https://arxiv.org/abs/2007.02133)

Masked Label Prediction: Unified Message Passing Model for Semi-Supervised Classification (https://arxiv.org/abs/2009.03509)

Combining Label Propagation and Simple Models Out-performs Graph Neural Networks (https://arxiv.org/abs/2010.13993)

大家也可以看看github的[ UniMP](https://github.com/PaddlePaddle/PGL/tree/main/ogb_examples/nodeproppred/unimp)算法 这个例子，里面有相似的数据集，并且最近也是SOTA效果，有帮助👏欢迎点Star

相关课程连接：[图神经网络7日打卡营](http://aistudio.baidu.com/aistudio/education/group/info/1956)